In [3]:
from torchvision.ops import masks_to_boxes
from torchvision.io import decode_image
import os
import torch
from torchvision.transforms.v2 import functional as F
from torchvision import tv_tensors
import sys

# 获取当前脚本所在的目录
current_dir = os.getcwd()
# 添加 detection 目录到 sys.path，以便解释器能够找到detection文件夹下的模块
sys.path.append(os.path.join(current_dir, 'detection'))


class SegmentationToDetectionDataset(torch.utils.data.Dataset):
    def __init__(self, data_dict, transform=None):
        super().__init__()
        self.data_dict = data_dict
        self.transforms = transform
        self.image_path_list = []
        self.mask_path_list = []
        # 遍历字典，整理数据
        for img_path, mask_path in data_dict.items():
            self.image_path_list.append(img_path)
            self.mask_path_list.append(tuple(mask_path))

    def __len__(self):
        return len(self.image_path_list)

    def __getitem__(self, idx):
        # load images and masks

        img_path = self.image_path_list[idx]
        mask_path_tuple = self.mask_path_list[idx]  # 同一张图片可能有良性和恶性的mask，所以tuple的大小可能为1或者2
        print(img_path)

        img = decode_image(img_path)  # 直接读取图片为tensor

        labels_list = []

        if len(mask_path_tuple) > 1:  # 如果图片的mask有两个
            masks_list = []
            for mask_path in mask_path_tuple:
                temp_mask = decode_image(mask_path)

                obj_ids = torch.unique(temp_mask)
                # first id is the background, so remove it
                obj_ids = obj_ids[1:]
                print(f"obj_ids: {obj_ids}")
                num_objs = len(obj_ids)

                if "malignant" in mask_path:
                    # 创建一个新的 tensor 来存储更新后的 mask
                    updated_mask = temp_mask.clone()  # 克隆原始 mask 以避免修改原数据
                    # 对于每个 obj_id，找到其在 mask 中的位置，并将其值设置为 100 + obj_id
                    for obj_id in obj_ids:
                        updated_mask[temp_mask == obj_id] = 100 + obj_id  # 避免恶性的mask值和良性的冲突，直接加100
                    masks_list.append(updated_mask)
                else:
                    masks_list.append(temp_mask)

                labels = torch.ones((num_objs,), dtype=torch.int64) if "benign" in mask_path else torch.ones((num_objs,), dtype=torch.int64) + 1  # 良性为1，恶性为2，背景为0
                labels_list.append(labels)
            # print(masks_list[1])
            mask = torch.max(masks_list[0], masks_list[1])  # 合并两个mask，如果两个mask有重叠的部分，则取最大值
        else:   # 如果图片的mask只有一个
            mask_path = mask_path_tuple[0]
            mask = decode_image(mask_path_tuple[0])
            obj_ids = torch.unique(mask)
            # first id is the background, so remove it
            obj_ids = obj_ids[1:]
            num_objs = len(obj_ids)

            labels = torch.ones((num_objs,), dtype=torch.int64) if "benign" in mask_path else torch.ones((num_objs,), dtype=torch.int64) + 1  # 良性为1，恶性为2，背景为0
            labels_list.append(labels)

        # 如果只有一个mask，则直接返回，如果有两个mask，则变量mask为合并后的mask，labels_list为两个mask对应的标签
        obj_ids = torch.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        print(f"obj_ids_after: {obj_ids}")

        masks = (mask == obj_ids[:, None, None]).to(dtype=torch.uint8)
        # get bounding box coordinates for each mask
        boxes = masks_to_boxes(masks)
        # print(labels_list)
        labels = torch.cat(labels_list)

        image_id = idx
        # 计算每个实例的面积
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((len(obj_ids),), dtype=torch.int64)
        # Wrap sample and targets into torchvision tv_tensors:
        img = tv_tensors.Image(img)

        target = {
            "boxes": tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=F.get_size(img)),
            "labels": labels,
            "masks": tv_tensors.Mask(masks),
            "image_id": image_id,
            "area": area,
            "iscrowd": iscrowd
        }

        # 过滤掉面积为0的实例
        non_zero_index = target["area"] != 0  # 获取非0面积的索引
        keys_to_filter = ["boxes", "labels", "masks", "area", "iscrowd"]
        filtered_target = {key: (value[non_zero_index] if key in keys_to_filter else value) for key, value in target.items()}

        if self.transforms is not None:
            img, filtered_target = self.transforms(img, filtered_target)

        return img, filtered_target

In [7]:
import json
# import matplotlib.pyplot as plt
#
with open("img_mask_path.json", "r") as f:
    data_dict = json.load(f)
dataset = SegmentationToDetectionDataset(data_dict)
# #
# image, target = dataset[4]
# print(image.shape, target["boxes"].shape, target["masks"].shape, target["iscrowd"])
# print(target["area"])
# print(target["labels"])
#
# print("=" * 50)

#
# print(image)
# print(mask.shape)
# plt.figure(figsize=(16, 8))
# plt.subplot(121)
# plt.title("Image")
# plt.imshow(image.permute(1, 2, 0))
# plt.subplot(122)
# plt.title("Mask")
# plt.imshow(mask.permute(1, 2, 0))


for i in range(12252, len(dataset)):
    print(i)
    img, target = dataset[i]
    print(img.shape, target["boxes"].shape, target["masks"].shape, target["iscrowd"])
    print(target["area"])
    print(target["labels"])
    print("=" * 50)


12252
G:/Program/DATABASE_JPG/2021/ChengSiMin/ChengSiMin_173.jpg
obj_ids: tensor([1], dtype=torch.uint8)
obj_ids: tensor([2], dtype=torch.uint8)
obj_ids_after: tensor([  1, 102], dtype=torch.uint8)
torch.Size([1, 512, 512]) torch.Size([2, 4]) torch.Size([2, 512, 512]) tensor([0, 0])
tensor([ 77., 380.])
tensor([1, 2])
12253
G:/Program/DATABASE_JPG/2023/ZhangLu/ZhangLu_87.jpg
obj_ids_after: tensor([1, 5], dtype=torch.uint8)
torch.Size([1, 512, 512]) torch.Size([2, 4]) torch.Size([2, 512, 512]) tensor([0, 0])
tensor([247.,  88.])
tensor([1, 1])
12254
G:/Program/DATABASE_JPG/2021/LaiLuNa/LaiLuNa_116.jpg
obj_ids_after: tensor([8], dtype=torch.uint8)
torch.Size([1, 512, 512]) torch.Size([1, 4]) torch.Size([1, 512, 512]) tensor([0])
tensor([196.])
tensor([1])
12255
G:/Program/DATABASE_JPG/2023/DingQiaoLing/DingQiaoLing_77.jpg
obj_ids_after: tensor([9], dtype=torch.uint8)
torch.Size([1, 512, 512]) torch.Size([1, 4]) torch.Size([1, 512, 512]) tensor([0])
tensor([70.])
tensor([1])
12256
G:/Prog

## Faster R-CNN resnet50_fpn
fasterrcnn_resnet50_fpn 是 PyTorch 的 torchvision 库中提供的一个用于目标检测的深度学习模型。它结合了 ResNet-50 作为骨干网络和 Feature Pyramid Network (FPN) 作为多尺度特征提取机制。下面是 fasterrcnn_resnet50_fpn 模型的主要组成部分及其功能的详细说明：

1. 骨干网络 (Backbone)
- ResNet-50: 这是一个深度残差网络，由多个残差块组成，每个残差块包含几个卷积层。ResNet-50 具有50层，分为5个阶段，每个阶段包含若干个残差块。ResNet-50 的主要作用是从输入图像中提取高级特征。

2. 特征金字塔网络 (Feature Pyramid Network, FPN)
- FPN：FPN 是一种多尺度特征提取机制，它利用不同层次的特征图来构建一个金字塔结构。FPN 通过自顶向下的路径和横向连接来融合不同层次的特征图，从而增强特征的丰富性。具体来说：
    - 自顶向下路径：从最高层次的特征图开始，通过上采样操作逐步生成低层次的特征图。
    - 横向连接：将自顶向下路径生成的特征图与同一层次的特征图进行逐元素相加，从而融合不同层次的信息。
3. 区域提议网络 (Region Proposal Network, RPN)
- RPN：这是一个用于生成候选区域（Region Proposals）的网络。RPN 在 FPN 的多尺度特征图上滑动一个小窗口，生成一组候选框（anchors），并为每个候选框预测两个值：
    - 对象分数：表示该候选框是否包含目标对象的概率。
    - 边界框回归：用于调整候选框的位置和大小，使其更准确地包围目标对象。
4. ROI Pooling
- ROI Pooling：在 RPN 生成的候选框基础上，ROI Pooling 将每个候选框映射到特征图上的一个固定大小的区域（候选框是基于原图的，而特征图原图的大小不一致，所以需要将候选框映射到特征图上）。然后，通过最大池化或平均池化操作，将这些区域转换为固定大小的特征向量（因为映射过来的ROI大小不一致[RPN会生成大量的候选框，这些候选框的大小和形状是根据预定义的锚点（anchor boxes）和图像特征动态生成的，因此会非常多样。]，而后续的后续的网络层（如全连接层）通常要求输入具有固定的尺寸，所以要进行池化）。
5. 目标检测头 (Detection Head)
- Box Predictor：这是模型的最后一部分，负责最终的目标检测任务。它包括两个主要组件：
    - 分类器 (Classifier)：对每个候选框进行分类，预测其属于各个类别的概率。
    - 边界框回归器 (BBox Regressor)：进一步微调每个候选框的位置和大小，使其更精确地包围目标对象。

### 模型结构总结
- 输入图像：输入一张图像。
- 骨干网络 (ResNet-50)：提取图像的高级特征。
- 特征金字塔网络 (FPN)：生成多尺度特征图。
- 区域提议网络 (RPN)：生成候选框并进行初步筛选。
- ROI Pooling：将候选框映射到固定大小的特征向量。
- 目标检测头 (Detection Head)：
- 分类器：对候选框进行分类。
- 边界框回归器：调整候选框的位置和大小。


## ROI池化
[ROI池化视频讲解](https://b23.tv/w0zRRKp)

ROI（Region of Interest）池化是一种在计算机视觉任务中常用的处理技术，特别是在目标检测和实例分割等需要从图像的不同区域提取特征的任务中。它主要应用于深度学习框架中的卷积神经网络（CNNs），用于将不同大小的输入区域转换成固定大小的输出。

### 作用

1. **统一尺寸**：在目标检测任务中，候选框（即感兴趣区域，ROIs）的大小和形状各不相同，但后续的全连接层需要固定大小的输入。ROI池化可以将这些不同大小的候选框转换为统一的尺寸，使得它们能够被送入到后续的网络层中进行处理。

2. **保留空间信息**：与全局平均池化或最大池化不同，ROI池化是在每个ROI内独立进行的，这样可以保留每个ROI的空间结构信息，这对于目标检测和分类非常重要。

3. **提高效率**：通过将不同大小的ROI转换为相同的尺寸，可以在一定程度上减少计算量，尤其是在处理大规模数据集时，这种效率提升尤为重要。

### 工作原理

ROI池化的基本思想是首先定义一个固定的输出大小（例如7x7），然后对于每一个输入的ROI，无论其原始大小如何，都将该ROI划分为与输出大小相匹配的子区域。接下来，在每个子区域内执行最大池化或平均池化等操作，最终得到固定大小的输出。这个过程确保了所有ROI都能以相同的形式进入后续的网络层，同时保留了各自的空间信息。

### 应用场景

- **目标检测**：如在Faster R-CNN、Mask R-CNN等模型中，ROI池化用于处理由区域提议网络（RPN）生成的不同大小的候选框。
- **实例分割**：在Mask R-CNN等模型中，除了对边界框进行分类和回归外，还需要生成与每个目标对应的分割掩码，此时ROI池化同样扮演着重要角色。

总之，ROI池化是现代计算机视觉模型中不可或缺的一部分，它帮助解决了特征图与不同大小候选框之间的适配问题，提高了模型的灵活性和性能。

In [2]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

"""
使用FasterRCNNPredictor替换原有box_predictor，这个新的预测器将使用从上一行获取的输入特征数，并且输出类别数设置为num_classes。这样做是为了让模型能够针对新的分类任务进行预测。
"""

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
print(model)
# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 3  # 2 class (benign[1], malignant[2]) + background[0]
# get number of input features for the classifier
# 获取了模型中ROI头（Region of Interest Head）的box预测器中分类层的输入特征数。这是为了确定新分类器应该有多少个输入特征
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
# 用一个新的FastRCNNPredictor对象替换了模型原有的box预测器。这个新的预测器将使用从上一行获取的输入特征数，并且输出类别数设置为num_classes。这样做是为了让模型能够针对新的分类任务进行预测。
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

print("=" * 50)
print(model)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [4]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

"""
构建一个自定义的 FasterRCNN 模型，使用 MobileNetV2 作为骨干网络，并自定义了区域提议网络（RPN）和感兴趣区域池化（ROI Pooling）的参数。
"""

print("=" * 50)
# 使用 mobilenet_v2 函数加载预训练的 MobileNetV2 模型，但只保留其特征提取部分（即 features）。这是因为我们只需要模型的特征提取能力，而不需要分类头部。
# mobilenet_v2 只有features和classifier两个模块，其中features是mobilenet_v2的主体，classifier是分类头部。
backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
# classifier = torchvision.models.mobilenet_v2(weights="DEFAULT").classifier


# 设置 backbone.out_channels 属性，告诉 FasterRCNN 骨干网络的输出通道数是多少。对于 MobileNetV2，这个值是 1280。
backbone.out_channels = 1280

# 创建一个 AnchorGenerator 实例，用于生成锚点（anchors）。锚点是在输入图像的不同位置生成的候选框，用于后续的目标检测任务。
# 这里配置了每个空间位置生成 5 种不同尺寸和 3 种不同宽高比的锚点。
anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256, 512),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

# FasterRCNN 自带的roi池化操作会丢失空间信息，所以需要使用 MultiScaleRoIAlign 代替。具体见上文roi池化视频讲解。
# 这里配置了使用第一个特征图（featmap_names=['0']），输出大小为 7x7，采样比率为 2。
roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    featmap_names=['0'],
    output_size=7,
    sampling_ratio=2
)

# put the pieces together inside a Faster-RCNN model
model = FasterRCNN(
    backbone,
    num_classes=3,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)
print(model)

# 以上的操作归纳如下：
# 1. 使用FasterRCNNPredictor替换原有box_predictor，使之能够针对新的分类任务进行预测。
# 2. 使用MobileNetV2作为骨干网络，
# 3. 使用自定义AnchorGenerator生成锚点（即候选框），并使用MultiScaleRoIAlign代替roi池化操作。
# 4. 构建FasterRCNN模型，并将前面两个步骤的结果作为输入。

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidu

ROI Align（Region of Interest Align）是目标检测和实例分割任务中的一种改进的池化方法，旨在解决传统ROI池化方法中存在的量化误差问题。ROI Align最初是在Mask R-CNN中引入的，用于提高模型的精度和性能。下面详细解释ROI Align的工作原理及其优势。

### 传统ROI Pooling的问题

在传统的ROI Pooling中，候选框（ROI）会被划分成若干个子区域，每个子区域通过最大池化或平均池化操作来生成固定大小的输出。然而，这种做法存在以下问题：

1. **量化误差**：在将浮点数坐标转换为整数坐标时，会产生量化误差。例如，一个浮点数坐标 (2.5, 3.5) 被量化为整数坐标 (2, 3)，这会导致信息的丢失。
2. **不连续的采样**：传统ROI Pooling在每个子区域内的采样点是离散的，这可能导致特征图上的不连续性，影响后续的特征提取和分类性能。

### ROI Align的工作原理

ROI Align通过以下步骤解决了这些问题：

1. **划分子区域**：
   - 首先，将ROI划分为若干个子区域。假设输出大小为 \( k \times k \)，则将ROI划分为 \( k \times k \) 个子区域。

2. **双线性插值**：
   - 对于每个子区域，计算其四个角点的精确浮点数坐标。
   - 在每个子区域内，选择一个或多个采样点。这些采样点的坐标是通过双线性插值计算出来的，而不是简单的量化为整数坐标。
   - 使用双线性插值从特征图中获取采样点的特征值。具体来说，对于每个采样点，找到其最近的四个像素点，并通过双线性插值计算出该采样点的特征值。

3. **池化操作**：
   - 在每个子区域内，对采样点的特征值进行池化操作（通常是最大池化或平均池化），生成固定大小的输出。

### 具体步骤

1. **输入**：
   - 输入是一个特征图 \( F \) 和一个ROI \( R \)，其中 \( R \) 的坐标为 \( (x_1, y_1, x_2, y_2) \)。

2. **划分子区域**：
   - 假设输出大小为 \( k \times k \)，则将ROI \( R \) 划分为 \( k \times k \) 个子区域。
   - 每个子区域的宽度和高度分别为 \( w/k \) 和 \( h/k \)，其中 \( w = x_2 - x_1 \) 和 \( h = y_2 - y_1 \)。

3. **采样点选择**：
   - 在每个子区域内，选择一个或多个采样点。这些采样点的坐标是通过双线性插值计算出来的。
   - 例如，对于一个子区域，可以选择其中心点作为采样点，中心点的坐标为 \( (x_c, y_c) \)，其中 \( x_c = x_{\text{start}} + (w/k)/2 \) 和 \( y_c = y_{\text{start}} + (h/k)/2 \)。

4. **双线性插值**：
   - 找到采样点 \( (x_c, y_c) \) 最近的四个像素点 \( P_1, P_2, P_3, P_4 \)。
   - 使用双线性插值公式计算采样点的特征值：
     \[
     V = (1 - a)(1 - b)P_1 + a(1 - b)P_2 + (1 - a)bP_3 + abP_4
     \]
     其中，\( a = x_c - \lfloor x_c \rfloor \) 和 \( b = y_c - \lfloor y_c \rfloor \)。

5. **池化操作**：
   - 在每个子区域内，对采样点的特征值进行池化操作（通常是最大池化或平均池化），生成固定大小的输出。

### 优势

1. **减少量化误差**：通过双线性插值，ROI Align避免了量化误差，保留了特征图上的连续性。
2. **提高精度**：更精确的采样和池化操作使得特征提取更加准确，从而提高模型的整体性能。
3. **平滑特征**：双线性插值使得特征图上的特征更加平滑，有助于后续的特征提取和分类任务。

### 总结

ROI Align通过引入双线性插值和更精确的采样方法，有效解决了传统ROI Pooling中存在的量化误差和不连续性问题，提高了目标检测和实例分割任务的精度和性能。这一技术在现代计算机视觉任务中得到了广泛应用，特别是在需要高精度特征提取的应用中。

In [5]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    """
    修改了原始模型的box_predictor和mask_predictor的输出类别数，从默认的91修改为了num_classes。
    """
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    # 默认的分类器就是FasterRCNNPredictor，只不过输出类别数是91，这里修改为num_classes。
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    # 默认的分类器就是MaskRCNNPredictor，只不过输出类别数是91，这里修改为num_classes。
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes
    )

    return model

In [6]:
from torchvision.transforms import v2 as T


def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

In [7]:
import json
import detection.utils as utils

with open("img_mask_path.json", "r") as f:
    data_dict = json.load(f)

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
dataset = SegmentationToDetectionDataset(data_dict, get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=utils.collate_fn
)

# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)  # Returns losses and detections
print(output)

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)  # Returns predictions
print(predictions[0])

{'loss_classifier': tensor(0.0845, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.0050, grad_fn=<DivBackward0>), 'loss_objectness': tensor(1.8251, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.1083, grad_fn=<DivBackward0>)}
{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>), 'labels': tensor([], dtype=torch.int64), 'scores': tensor([], grad_fn=<IndexBackward0>)}


In [8]:
from detection.engine import train_one_epoch, evaluate


# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 3
# use our dataset and defined transformations
dataset = SegmentationToDetectionDataset(data_dict, get_transform(train=True))
dataset_test = SegmentationToDetectionDataset(data_dict, get_transform(train=False))

# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=utils.collate_fn
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    collate_fn=utils.collate_fn
)

# get the model using our helper function
model = get_model_instance_segmentation(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

# let's train it just for 2 epochs
num_epochs = 1

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

print("That's it!")

[{'boxes': tensor([[122., 283., 136., 297.],
        [143., 200., 159., 217.],
        [147., 213., 159., 227.],
        [370., 209., 384., 223.],
        [132., 210., 151., 227.],
        [359., 221., 391., 263.]], device='cuda:0'), 'labels': tensor([1, 1, 1, 1, 1, 1], device='cuda:0'), 'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ...,

G:\Program\DeepLearning\detection\engine.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch: [0]  [    0/18473]  eta: 3:17:26  lr: 0.000010  loss: 3.5770 (3.5770)  loss_classifier: 0.9037 (0.9037)  loss_box_reg: 0.0818 (0.0818)  loss_mask: 1.5998 (1.5998)  loss_objectness: 0.9488 (0.9488)  loss_rpn_box_reg: 0.0428 (0.0428)  time: 0.6413  data: 0.0148  max mem: 867
[{'boxes': tensor([[132., 327., 147., 347.],
        [140., 318., 151., 333.],
        [108., 269., 331., 348.],
        [104., 243., 112., 254.]], device='cuda:0'), 'labels': tensor([1, 1, 2, 2], device='cuda:0'), 'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
        

IndexError: The shape of the mask [5] at index 0 does not match the shape of the indexed tensor [6] at index 0